## CH02

In [1]:
import torch

In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
dataset = pd.read_csv("data/car_evaluation.csv")

In [4]:
dataset.head()

,price,maint,doors,persons,lug_capacity,safety,output
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [5]:
categorical_columns = ["price", "maint", "doors", "persons", "lug_capacity", "safety"]

In [6]:
for category in categorical_columns:
    dataset[category] = dataset[category].astype("category")

In [7]:
price = dataset["price"].cat.codes.values
maint = dataset["maint"].cat.codes.values
doors = dataset["doors"].cat.codes.values
persons = dataset["persons"].cat.codes.values
lug_capacity = dataset["lug_capacity"].cat.codes.values
safety = dataset["safety"].cat.codes.values

categorical_data = np.stack([price, maint, doors, persons, lug_capacity, safety], 1)
categorical_data[:10]

array([[3, 3, 0, 0, 2, 1],
       [3, 3, 0, 0, 2, 2],
       [3, 3, 0, 0, 2, 0],
       [3, 3, 0, 0, 1, 1],
       [3, 3, 0, 0, 1, 2],
       [3, 3, 0, 0, 1, 0],
       [3, 3, 0, 0, 0, 1],
       [3, 3, 0, 0, 0, 2],
       [3, 3, 0, 0, 0, 0],
       [3, 3, 0, 1, 2, 1]], dtype=int8)

In [8]:
categorical_data = torch.tensor(categorical_data, dtype=torch.int64)
categorical_data[:10]

tensor([[3, 3, 0, 0, 2, 1],
        [3, 3, 0, 0, 2, 2],
        [3, 3, 0, 0, 2, 0],
        [3, 3, 0, 0, 1, 1],
        [3, 3, 0, 0, 1, 2],
        [3, 3, 0, 0, 1, 0],
        [3, 3, 0, 0, 0, 1],
        [3, 3, 0, 0, 0, 2],
        [3, 3, 0, 0, 0, 0],
        [3, 3, 0, 1, 2, 1]])

In [9]:
outputs = pd.get_dummies(dataset.output)
outputs = outputs.values
outputs = torch.tensor(outputs).flatten()

print(categorical_data.shape)
print(outputs.shape)

torch.Size([1728, 6])
torch.Size([6912])


In [10]:
categorical_column_sizes = [
    len(dataset[column].cat.categories) for column in categorical_columns
]


categorical_embedding_sizes = [
    (col_size, min(50, (col_size + 1) // 2)) for col_size in categorical_column_sizes
]


print(categorical_embedding_sizes)

[(4, 2), (4, 2), (4, 2), (3, 2), (3, 2), (3, 2)]


In [11]:
total_records = 1728
test_records = int(total_records * 0.2)

categorical_train_data = categorical_data[: total_records - test_records]
categorical_test_data = categorical_data[total_records - test_records : total_records]
train_outputs = outputs[: total_records - test_records]
test_outputs = outputs[total_records - test_records : total_records]

In [12]:
print(len(categorical_train_data))
print(len(train_outputs))
print(len(categorical_test_data))
print(len(test_outputs))

1383
1383
345
345


In [13]:
class Model(nn.Module):
    def __init__(self, embedding_size, output_size, layers, p=0.4):
        super().__init__()
        self.all_embeddings = nn.ModuleList(
            [nn.Embedding(ni, nf) for ni, nf in embedding_size]
        )
        self.embedding_dropout = nn.Dropout(p)
        all_layers = []
        num_categorical_cols = sum((nf for ni, nf in embedding_size))
        input_size = num_categorical_cols
        for i in layers:
            all_layers.append(nn.Linear(input_size, i))
            all_layers.append(nn.ReLU(inplace=True))
            all_layers.append(nn.BatchNorm1d(i))
            all_layers.append(nn.Dropout(p))
            input_size = i
        all_layers.append(nn.Linear(layers[-1], output_size))
        self.layers = nn.Sequential(*all_layers)

    def forward(self, x_categorical):
        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        x = torch.cat(embeddings, 1)
        x = self.embedding_dropout(x)
        x = self.layers(x)
        return x

In [14]:
model = Model(categorical_embedding_sizes, 4, [200, 100, 50], p=0.4)
print(model)

Model(
  (all_embeddings): ModuleList(
    (0-2): 3 x Embedding(4, 2)
    (3-5): 3 x Embedding(3, 2)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=12, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=100, out_features=50, bias=True)
    (9): ReLU(inplace=True)
    (10): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.4, inplace=False)
    (12): Linear(in_features=50, out_features=4, bias=True)
  )
)


In [15]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [16]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [17]:
epochs = 500
aggregated_losses = []
train_outputs = train_outputs.to(device=device, dtype=torch.int64)
for i in range(epochs):
    i += 1
    y_pred = model(categorical_train_data)
    single_loss = loss_function(y_pred, train_outputs)
    aggregated_losses.append(single_loss)

    if i % 25 == 1:
        print(f"epoch: {i:3} loss: {single_loss.item():10.8f}")

    optimizer.zero_grad()
    single_loss.backward()
    optimizer.step()

print(f"epoch: {i:3} loss: {single_loss.item():10.10f}")

epoch:   1 loss: 1.57561541
epoch:  26 loss: 1.36146855
epoch:  51 loss: 1.29612863
epoch:  76 loss: 1.16816628
epoch: 101 loss: 1.06585729
epoch: 126 loss: 0.93245596
epoch: 151 loss: 0.84296274
epoch: 176 loss: 0.74279982
epoch: 201 loss: 0.70231849
epoch: 226 loss: 0.64878613
epoch: 251 loss: 0.63121897
epoch: 276 loss: 0.60157251
epoch: 301 loss: 0.58657551
epoch: 326 loss: 0.58692664
epoch: 351 loss: 0.57182604
epoch: 376 loss: 0.57363534
epoch: 401 loss: 0.56346190
epoch: 426 loss: 0.55488002
epoch: 451 loss: 0.56109756
epoch: 476 loss: 0.55321872
epoch: 500 loss: 0.5413835049


In [22]:
test_outputs = test_outputs.to(device="cpu", dtype=torch.int64)
with torch.no_grad():
    y_val = model(categorical_test_data)
    loss = loss_function(y_val, test_outputs)
print(f"Loss: {loss:.8f}")

Loss: 0.51059890


In [23]:
print(y_val[:5])

tensor([[ 1.5933,  0.6639, -2.4132, -2.0994],
        [ 2.6221,  1.3876, -3.4371, -3.3311],
        [ 2.8872,  1.6997, -3.4538, -3.3943],
        [ 1.3479,  0.6112, -1.6605, -1.5028],
        [ 2.7005,  2.3741, -2.7024, -2.4496]])


In [24]:
y_val = np.argmax(y_val, axis=1)
print(y_val[:5])

tensor([0, 0, 0, 0, 0])


In [26]:
!conda install anaconda::scikit-learn -y

Channels:
 - defaults
 - anaconda
 - conda-forge
 - pytorch
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\user\anaconda3\envs\torch-book

  added / updated specs:
    - anaconda::scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    icc_rt-2022.1.0            |       h6049295_2         6.5 MB
    joblib-1.4.2               |  py312haa95532_0         515 KB
    pybind11-abi-5             |       hd3eb1b0_0          14 KB
    scikit-learn-1.3.0         |  py312hc7c4135_2         9.4 MB  anaconda
    scipy-1.13.1               |  py312hbb039d4_0        22.5 MB
    threadpoolctl-3.5.0        |  py312hfc267ef_0          49 KB
    ------------------------------------------------------------
                                           Total:        39.0 MB

The following NEW packages will be INSTALLED:

  icc_rt           

In [27]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(test_outputs, y_val))
print(classification_report(test_outputs, y_val))
print(accuracy_score(test_outputs, y_val))

[[258   1]
 [ 79   7]]
              precision    recall  f1-score   support

           0       0.77      1.00      0.87       259
           1       0.88      0.08      0.15        86

    accuracy                           0.77       345
   macro avg       0.82      0.54      0.51       345
weighted avg       0.79      0.77      0.69       345

0.7681159420289855
